In [1]:
from stlist import stList
import requests as rq
from bs4 import BeautifulSoup as bs
import time as t
import random
import os
import pandas as pd
import sqlite3 as sql
import datetime


### 建立與 CWB CODiS 查詢頁面連線，將頁面中的測站資料爬下，與現有測站資料比對，
#### 若有新測站，則更新現有檔案

In [ ]:
st_http = "http://e-service.cwb.gov.tw/HistoryDataQuery/QueryDataController.do?command=viewMain&_=1473065277608"

soup = bs(rq.get(st_http).text, "html.parser")

list_text = soup.find("script",type="text/javascript").text
head = list_text.find("var stList =") + 13
end = list_text[head:].find("}")
var = bs(list_text[head:head+end+1],"html.parser")
newList = eval(var.text)

for key in newList.keys():
    for i in range(len(newList[key])):
        newList[key][i] = newList[key][i].strip()

#print(newList)                                                                                                                              
if newList != stList:
    os.rename("./stlist.py", "./stlist_old.py")
    with open("./stlist.py", "w+") as fw:
        fw.write("stList=")
        fw.write(str(newList))



#### 將所有測站從 2013/1/1~2016/8/31 的資料爬下寫入資料夾
##### 檔名為 <編號>_<測站ID>_<站名>_<縣市>/<日期>_<編號>_<測站ID>_<站名>.html

In [ ]:
http = "http://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=%s&stname=%s&datepicker=%d-%d"

proxies = {
    'https': 'http://220.141.162.136:8080',
}

#z = 0                                                                                                                                       
stl = list(stList.keys())
def get_data(ID):
    z = stl.index(ID)
    StN_C = stList[ID][0]
    StN = stList[ID][1]
    County = stList[ID][2]
    directory = "./db_data/%d_%s_%s_%s/" % (z,ID,StN_C, County)
    if not os.path.exists(directory):
        os.makedirs(directory)
    for year in [2013, 2014, 2015, 2016]:
        for month in range(1,13):
            if not (year == 2016 and month in range(9,13)):
                f = directory+"%d-%d_%s_%s.html" % (year, month, ID, StN)
                if not os.path.exists(f):
                    t.sleep(3)
                    html = rq.get(http % (ID, StN, year, month), proxies=proxies)
                    t.sleep(random.randint(0,5))
                    with open(f , "w+") as fw:
                        fw.write(html.text)
                    #end write                                                                                                               
                #end file                                                                                                                    
    #end                  

jobs = [get_data(ID) for ID in stList.keys()]


#### 將 html 中需要的資料寫入 .db 檔
##### 以年為 table 
##### 檔名為 <編號><測站ID><站名><縣市>.db

In [ ]:
stl = list(stList.keys())
db_path = "./db/%s.db"
target_dir = [i for i in os.listdir("./data") if os.path.isdir("./data/"+i)]

def make_db(directory):
    for _html in sorted(os.listdir("./data/"+directory), key=lambda x: datetime.datetime.strptime(x[:6] if x[6]=="_" else x[:7], '%Y-%m')):
        year = _html[:4]
        date = _html[:6] if _html[6]=='_' else _html[:7]
        with open("./data/%s/%s" % (directory, _html), "r+") as html:
            sp = bs(html, "lxml")
        sp.tbody.name = "table"
        table = sp.find_all("table")[1]
        table.table.tr.replace_with("")
        Df = pd.read_html(str(table.table), flavor="bs4", header=0)[0]
        data = Df[["觀測時間(LST)ObsTime","測站氣壓(hPa)StnPres","氣溫(℃)Temperature","最高氣溫(℃)T Max","最低氣溫(℃)T Min","相對溼度(%)R\
H","風速(m/s)WS","降水量(mm)Precp"]]
        data["觀測時間(LST)ObsTime"] = data["觀測時間(LST)ObsTime"].apply(lambda x: date+"-"+str(x))
        db = sql.connect(db_path % directory)
        data.to_sql(year, db, if_exists="append", index=False)
        db.close()


jobs = [make_db(directory) for directory in target_dir]


In [5]:
ls

README.rst
__pycache__/
data/
db/
db_data/
step 3 -- SVM 東方果實蠅預測模型_新_PLOT_BOX + Test with PCA.ipynb
step 5--東方果實蠅預測模型-dia-Copy1-pest.ipynb
step 5--東方果實蠅預測模型-dia-Copy1.ipynb
step0_build_database.ipynb
step1__抓天氣資料_氣溫3降水量.ipynb
step2__整理資料.ipynb
step4.5__抓最近九天的天氣資料-更新每天早上資料.ipynb
step4__抓最近九天的天氣資料.ipynb
stlist.py
